In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [6]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "AvgDrawdown", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio", "TheDfactor"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

XRP-PERP: 100%|██████████| 32/32 [01:39<00:00,  3.11s/it]  


In [10]:
analysis_df = analysis_df.round(2)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [11]:
pd.read_csv(ANALYSIS_FILEPATH).sort_values("TheDfactor", ascending=False) 

,Name,AvgReturns,StdDev,AvgDrawdown,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio,TheDfactor
26,TRU-PERP,19.43,38.36,7.81,0.51,-9.03,99.81,-3.45,76.62,22.18,-0.28,-1.39,76.62,55.27,1.01
18,LUNA-PERP,70.74,88.54,5.64,0.80,1.04,195.68,0.00,70.74,70.74,15.51,0.00,15.51,15.51,0.92
30,XRP-PERP,16.77,21.74,9.34,0.77,-1.67,58.13,-1.16,30.21,26.05,6.90,-1.04,27.90,26.84,0.41
25,SXP-PERP,34.92,63.03,4.55,0.55,-5.80,159.84,-3.69,60.66,16.43,2.92,-3.69,19.22,5.21,0.23
5,BNB-PERP,23.74,32.65,8.16,0.73,1.24,88.08,0.00,23.74,23.74,8.65,0.00,8.65,8.65,0.21
16,LINK-PERP,17.24,26.39,7.11,0.65,-5.19,77.29,-3.24,32.60,10.06,16.63,-2.49,18.25,7.34,0.15
20,RUNE-PERP,13.27,24.17,9.02,0.55,-13.93,61.78,-5.23,36.38,6.96,-0.52,-3.69,33.44,9.05,0.11
28,UNI-PERP,22.74,23.48,6.79,0.97,-3.24,82.44,-3.24,25.99,8.03,18.36,-3.24,21.32,6.59,0.11
8,DOGE-PERP,25.86,32.80,10.72,0.79,-12.39,82.25,-6.26,47.28,7.55,25.99,-6.26,33.59,5.36,0.10
4,BCH-PERP,27.53,45.30,8.35,0.61,-5.38,116.46,-4.29,48.75,11.36,12.94,-4.29,16.85,3.93,0.09
